In [1]:
import textwrap,json,zlib,copy,pickle,math,codecs
import multihash,ipfsapi
from solc import compile_source, compile_files, link_code
from web3 import Web3, KeepAliveRPCProvider
from os.path import isfile, join
from os import listdir
import numpy as np
import phe as paillier

In [2]:
class Vector():
    
    def __init__(self,value):
        super(type(self))
        self.value = np.array(value)
        
    def encrypt(self,pubkey):
        self.pubkey = pubkey
        self.value = np.array([self.pubkey.encrypt(x) for x in self.value])
        return self
    
    def decrypt(self,prikey):
        self.pubkey = None
        self.value = np.array([prikey.decrypt(x) for x in self.value])
        return self
        
    def __add__(self,x):
        v = copy.deepcopy(self)
        v.value += x.value
        return v
    
    def __sub__(self,x):
        v = copy.deepcopy(self)
        v.value -= x.value
        return v
    
    def __mul__(self,scalar):
        v = copy.deepcopy(self)
        v.value *= scalar
        return v
    
    def __div__(self,scalar):
        v = copy.deepcopy(self)
        v.value /= scalar
        return v
    
    def __getitem__(self,i):
        return self.value[i]
    
    def __str__(self):
        return str(self.value)
    
    def __repr__(self):
        return repr(self.value)
    
class LinearClassifier():
    
    def __init__(self,n_inputs=4,n_labels=2,desc=""):
        super(type(LinearClassifier))
        
        self.desc = desc
        
        self.n_inputs = n_inputs
        self.n_labels = n_labels
        
        self.weights = list()
        for i in range(n_inputs):
            self.weights.append(Vector(np.zeros(n_labels).astype('float64')))
        
        self.pubkey = None
    
    def encrypt(self,pubkey):
        self.pubkey = pubkey
        for w in self.weights:
            w.encrypt(pubkey)
        return self
            
    def decrypt(self,prikey):
        for w in self.weights:
            w.decrypt(prikey)
        self.pubkey = None            
        return self
    
    def forward(self,input=np.array([0,1,1,0])):
        
        pred = self.weights[0] * input[0]
        for j,each_inp in enumerate(input[1:]):
            if(each_inp == 1):
                pred = pred + self.weights[j+1]
            elif(each_inp != 0):
                pred = pred + (self.weights[j+1] * input[j+1])
                
        return pred
   
    def learn(self,input=np.array([0,1,1,0]),target=np.array([0,1]),alpha=0.5):
        target = np.array(target).astype('float64')
        pred = self.forward(input)

        target_v = Vector(target)
        
        if(self.pubkey is not None):
            target_v = target_v.encrypt(self.pubkey)
        
        grad = (pred - target_v) * alpha

        for i in range(len(input)):
            if(input[i] != 1 and input[i] != 0):
                self.weights[i] = self.weights[i] - (grad * input[i])
            elif(input[i] == 1):
                self.weights[i] = self.weights[i] - grad
            else:
                "doesn't matter... input == 0"
        
        return grad
    
    def __str__(self):
        return "Linear Model ("+str(self.n_inputs)+","+str(self.n_labels)+"): " + str(self.desc)
    
    def __repr__(self):
        return self.__str__()

In [3]:
class ModelMine():
    
    def __init__(self,account=None,deploy_txn=None,web3_port=8545,ipfs_port=5001):
        
        self.deploy_txn = deploy_txn
        self.web3 = Web3(KeepAliveRPCProvider(host='localhost', port=str(web3_port)))
        self.ipfs = ipfsapi.connect('127.0.0.1', int(ipfs_port))
        
        self.compile_and_deploy()
        if(account is not None):
            self.account = account
        else:
            print("No account submitted... using default[2]")
            self.account = self.web3.eth.accounts[2]
            
    def compile_and_deploy(self,directory='contracts/'):
        
        f = open('contracts/ModelMine.sol','r')
        source = f.read()
        f.close()
        
        compiled = compile_source(source)['<stdin>:ModelMine']

        contract = self.web3.eth.contract(
            abi = compiled['abi'],
            bytecode = compiled['bin'],
            bytecode_runtime = compiled['bin-runtime'],
            source = source,
            )
        
        if(self.deploy_txn is None):
            self.deploy_txn = contract.deploy()
        txn_receipt = self.web3.eth.getTransactionReceipt(self.deploy_txn)
        contract_address = txn_receipt['contractAddress']

        self.transact = contract.transact({
            "from":self.web3.eth.accounts[2],
            "to":contract_address,
            })

        self.call = contract.call({
            "from":self.web3.eth.accounts[2],
            "to":contract_address,
            })
        
        return self.deploy_txn

    
    def submit_model(self,model):
        ipfs_address = self.ipfs.add_pyobj(model)
        deploy_trans = self.transact.addModel([ipfs_address[0:32],ipfs_address[32:]])
        return self.call.getNumModels()-1
    
    def __getitem__(self,model_id):
        if(model_id < len(self)):
            mca = self.call.getModel(model_id)
            model_client = self.ipfs.get_pyobj(str(mca[0]+mca[1]).split("\x00")[0])
            return model_client
    
    def __len__(self):
        return self.call.getNumModels()

In [4]:
mine = ModelMine()

No account submitted... using default[2]


In [3]:
print("Generating paillier keypair")
pubkey, prikey = paillier.generate_paillier_keypair(n_length=2048*2)

# create model and upload to blockchain
# model = LinearClassifier(desc="Binary classification model.").encrypt(pubkey)
# model_id = mine.submit_model(model)

Generating paillier keypair


In [5]:
ct = pubkey.encrypt(1234)

In [6]:
prikey.decrypt(ct)

1234

In [6]:
model

Linear Model (4,2): Binary classification model.

In [7]:
model_client = mine[model_id]

for iter in range(10):
    print(iter)
    model_client.learn([0,0,0,1],np.array([0,1]).astype('float64'))
#     model.decrypt(prikey).encrypt(pubkey)

updated_id = mine.submit_model(model_client)

0
1
2
3
4
5
6
7
8
9


In [154]:
# download newly trained model, decrypt it and predict
model_server = mine[updated_id].decrypt(prikey)
pred = model_server.forward(input=[0,0,0,1])
pred

array([ 0.        ,  0.99902344])

In [ ]:
len(mine)

In [125]:
mine2 = ModelMine(mine.deploy_txn)